In [1]:
from torchHMM.model.discretized_HMM import DiscreteHMM, DISCRETIZATION_TECHNIQUES
from torchHMM.model.discretized_flow_HMM import FlowHMM

In [2]:
import numpy as np
import pandas as pd
import pickle as pkl

In [3]:
from hmmlearn.hmm import CategoricalHMM, GaussianHMM
from gensim.models import KeyedVectors


In [4]:
with open('clickstream_experiment/data/preprocessed_data/ClickStream_test_cleaned.pkl', 'rb') as f:
    cs = pkl.load(f)

In [5]:
session_train, sessions_test = cs.sessions[:-100000], cs.sessions[-100000:]


In [6]:
item_ids = np.unique([e.item_id for s in session_train for e in s.event_list])
item_ids = {item_ids[i]: i for i in range(len(item_ids))}

Xs = [[item_ids[e.item_id] for e in s.event_list] for s in session_train]
l = [len(y) for y in Xs]

X = np.concatenate(Xs).reshape(-1, 1)

target =  [[item_ids[e.item_id] for e in s.event_list if e.item_id in item_ids.keys()] for s in sessions_test]
target = [l for l in target if len(l)]
lt = np.array([len(y) for y in target])
target = np.concatenate(target).reshape(-1, 1)

In [7]:
len(item_ids.keys())

237227

In [8]:
chmm = CategoricalHMM(5, n_iter=10, verbose=True)


In [9]:
%time chmm.fit(X, l)

         1   -49052581.2711             +nan
         2   -44621579.9685    +4431001.3025
         3   -44480271.2162     +141308.7524
         4   -44369640.9999     +110630.2163
         5   -44235283.4194     +134357.5805
         6   -44054470.9312     +180812.4882
         7   -43832260.8200     +222210.1112
         8   -43593110.2618     +239150.5581
         9   -43367025.2830     +226084.9788


CPU times: user 10min 9s, sys: 230 ms, total: 10min 9s
Wall time: 10min 10s


        10   -43172983.8238     +194041.4593


CategoricalHMM(n_components=5,
               random_state=RandomState(MT19937) at 0x7FA6BFDBCA40,
               verbose=True)

In [10]:
print(chmm.score(target, lt))

-8799472.712577958


In [11]:
vectors = KeyedVectors.load(
    f"clickstream_experiment/data/preprocessed_data/vectors_train_20_10_5_8_cleaned.kv"
)
vecs = np.concatenate(
    [
        vectors.get_vector(k).reshape(1, -1)
        for k in list(vectors.key_to_index.keys())
    ]
)

In [12]:
unknown = vecs.mean(axis=0).reshape(1, -1)

def get_vec(e):
    try:
        return vectors.get_vector(e.item_id).reshape(1, -1)
    except:
        return unknown

In [13]:
Xs_c = [[get_vec(e) for e in s.event_list] for s in session_train]
l_c = np.array([len(y) for y in X])

X_c = np.concatenate([a for as_ in Xs_c for a in as_], axis=0)

target_c =  [[get_vec(e) for e in s.event_list if e.item_id in item_ids.keys()] for s in sessions_test]
target_c = np.concatenate([a for as_ in target_c for a in as_], axis=0)


In [14]:
w2vhmm = GaussianHMM(5, n_iter=10, verbose=True)

In [15]:
%time w2vhmm.fit(X_c, l_c)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
         1   -87556753.4024             +nan
         2   -66821266.8097   +20735486.5927
         3   -50855598.1229   +15965668.6868
         4   -43601801.7819    +7253796.3410
         5   -38613513.2108    +4988288.5711
         6   -34771092.4685    +3842420.7423
         7   -32818463.3727    +1952629.0958
         8   -31718938.6988    +1099524.6739
         9   -30952153.1754     +766785.5234
        10   -30388792.9051     +563360.2703
Some rows of transmat_ have zero sum because no transition from the state was ever observed.


CPU times: user 1h 20min 36s, sys: 7.66 s, total: 1h 20min 44s
Wall time: 1h 19min 58s


GaussianHMM(n_components=5, verbose=True)

In [16]:
w2vhmm.transmat_

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [17]:
zeros = w2vhmm.transmat_.sum(axis=1) == 0
w2vhmm.transmat_[zeros] = np.ones(w2vhmm.transmat_.shape)[zeros] / w2vhmm.n_components


In [18]:
print(w2vhmm.score(target_c, lt))

-5970890.821145764


In [19]:
w2v_dhmm = DiscreteHMM(n_components=5, no_nodes=64, optimizer="Adam", verbose=True, covariance_type='diag')

In [20]:
%time w2v_dhmm.fit(X_c, l_c)

/ziob/klaudia/miniconda3/envs/preproc/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/ziob/klaudia/FlowHMM/torchHMM/model/discretized_HMM.py:447: RuntimeWarning: invalid value encountered in divide
  cooc_matrix /= cooc_matrix.sum()


ValueError: 'diag' covars must have shape (n_components, n_dim)

In [21]:
print(w2v_dhmm.score(target_c, lt))

nan


In [22]:
w2v_fhmm = FlowHMM(n_components=5, no_nodes=512, l=100, optimizer="Adam", verbose=True)

%time w2v_fhmm.fit(X_c, l_c)

print(w2v_fhmm.score(target_c, lt))


MemoryError: Unable to allocate 149. GiB for an array with shape (3917265, 20, 512) and data type float32

TypeError: FlowHMM.discretize() missing 1 required positional argument: 'force'